# LightGBM - Reducción de Overfitting

## Objetivo
Probar LightGBM como alternativa a SVM, Random Forest y XGBoost para reducir overfitting manteniendo F1-score > 0.55.

## Ventajas de LightGBM
- ✅✅✅ Más rápido que XGBoost
- ✅✅✅ Regularización incorporada (reg_alpha L1, reg_lambda L2)
- ✅✅✅ Mejor control de overfitting con num_leaves
- ✅✅✅ Muy potente con datasets pequeños
- ✅✅✅ Subsampling y colsample_bytree reducen overfitting
- ✅✅✅ Maneja mejor el overfitting que XGBoost en algunos casos
- ✅ Menos propenso a F1=0 que SVM


## 1. Importación de librerías


In [1]:
import pandas as pd
import numpy as np
import pickle
import random

import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold
import optuna

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix
)

np.random.seed(42)
random.seed(42)

print("✅ Librerías importadas")


✅ Librerías importadas


## 2. Carga de datos


In [2]:
# Cargar datos
df = pd.read_csv('../data/processed/youtoxic_english_1000_processed.csv')
with open('../data/processed/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open('../data/processed/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

X_train_text = df[df.index.isin(range(len(y_train)))]['Text_processed'].values
X_test_text = df[df.index.isin(range(len(y_train), len(y_train) + len(y_test)))]['Text_processed'].values

print(f"✅ Datos cargados: {len(X_train_text)} train, {len(X_test_text)} test")
print(f"Distribución train: {np.bincount(y_train)}")
print(f"Distribución test: {np.bincount(y_test)}")


✅ Datos cargados: 800 train, 200 test
Distribución train: [430 370]
Distribución test: [108  92]


## 3. Vectorización


In [3]:
# Vectorización mejorada
tfidf = TfidfVectorizer(
    max_features=800,        # Más features
    ngram_range=(1, 2),      # Bigramas
    min_df=3,                # Menos restrictivo
    max_df=0.85,             # Más permisivo
    stop_words='english',
    sublinear_tf=True,
    norm='l2'
)

# SIN augmentación (LightGBM maneja bien dataset pequeño)
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

# LightGBM puede trabajar con sparse, pero para datasets pequeños es más rápido en denso
X_train_tfidf_dense = X_train_tfidf.toarray()
X_test_tfidf_dense = X_test_tfidf.toarray()

print(f"✅ Vectorización: {X_train_tfidf.shape[1]} features")
print(f"   Train shape: {X_train_tfidf_dense.shape}")
print(f"   Test shape: {X_test_tfidf_dense.shape}")


✅ Vectorización: 800 features
   Train shape: (800, 800)
   Test shape: (200, 800)


## 4. Función de Evaluación


In [4]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    """Evalúa modelo y retorna métricas."""
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_f1 = f1_score(y_train, y_train_pred, zero_division=0)
    test_f1 = f1_score(y_test, y_test_pred, zero_division=0)
    diff_f1 = abs(train_f1 - test_f1) * 100
    
    return {
        'train_f1': train_f1,
        'test_f1': test_f1,
        'test_accuracy': accuracy_score(y_test, y_test_pred),
        'test_precision': precision_score(y_test, y_test_pred, zero_division=0),
        'test_recall': recall_score(y_test, y_test_pred, zero_division=0),
        'diff_f1': diff_f1,
        'confusion_matrix': confusion_matrix(y_test, y_test_pred)
    }


## 5. Función Objetivo para Optuna


In [5]:
def objective(trial):
    """
    Función objetivo para LightGBM:
    - Regularización incorporada (reg_alpha L1, reg_lambda L2)
    - Control de overfitting con num_leaves y max_depth
    - Subsampling y colsample_bytree
    - Prioriza overfitting <5% y F1 >0.55
    """
    # Calcular scale_pos_weight para balance de clases
    scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])
    
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_leaves': trial.suggest_int('num_leaves', 10, 50),  # Controla complejidad
        'max_depth': trial.suggest_int('max_depth', 3, 10),  # Limita profundidad
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),  # Controla hojas
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),  # Subsampling de filas
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),  # Subsampling de features
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),  # Regularización L1
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),  # Regularización L2
        'scale_pos_weight': scale_pos_weight,  # Balance de clases
        'random_state': 42,
        'verbose': -1,  # Silenciar output
        'n_jobs': -1
    }
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train_tfidf_dense, y_train)
    
    results = evaluate_model(model, X_train_tfidf_dense, X_test_tfidf_dense, y_train, y_test)
    
    # Rechazar modelos inútiles
    if results['test_f1'] < 0.55:
        return -10.0
    
    # Rechazar overfitting extremo
    if results['diff_f1'] > 6.0:
        return -20.0
    
    # Rechazar recall extremo
    if results['test_recall'] >= 0.95:
        return -15.0
    
    # PRIORIDAD 1: Control de overfitting
    if results['diff_f1'] < 5.0:
        overfitting_bonus = (5.0 - results['diff_f1']) * 0.50  # Bonus grande
    else:
        overfitting_bonus = 0
    
    # PRIORIDAD 2: Penalización por overfitting
    if results['diff_f1'] > 5.0:
        overfitting_penalty = ((results['diff_f1'] - 5.0) ** 2) * 0.05
    else:
        overfitting_penalty = 0
    
    # PRIORIDAD 3: Penalizar recall extremo
    recall_penalty = 0
    if results['test_recall'] > 0.80:
        recall_penalty = ((results['test_recall'] - 0.80) ** 2) * 0.40
    
    # PRIORIDAD 4: F1-score base
    base_score = results['test_f1'] * 0.3
    
    score = base_score + overfitting_bonus - overfitting_penalty - recall_penalty
    return score

print("✅ Función objetivo definida (prioriza overfitting <5%)")


✅ Función objetivo definida (prioriza overfitting <5%)


## 6. Optimización con Optuna


In [6]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))

print("="*80)
print("OPTIMIZACIÓN LIGHTGBM - CONTROL DE OVERFITTING")
print("="*80)
print("✅ SIN augmentación (LightGBM maneja bien dataset pequeño)")
print("✅ Regularización incorporada (reg_alpha L1, reg_lambda L2)")
print("✅ Control de complejidad (num_leaves, max_depth)")
print("✅ Subsampling (subsample, colsample_bytree)")
print("✅ Scale pos weight para balance de clases")
print("✅ Penalización por overfitting >5%")
print("\nObjetivo: F1 > 0.55 Y overfitting < 5%")
print("Trials: 200")
print("-"*80)

study.optimize(objective, n_trials=200, show_progress_bar=True)

print("\n✅ Optimización completada")


[I 2025-12-04 08:26:11,789] A new study created in memory with name: no-name-7de79675-531f-4457-9540-ab706ac7f514


OPTIMIZACIÓN LIGHTGBM - CONTROL DE OVERFITTING
✅ SIN augmentación (LightGBM maneja bien dataset pequeño)
✅ Regularización incorporada (reg_alpha L1, reg_lambda L2)
✅ Control de complejidad (num_leaves, max_depth)
✅ Subsampling (subsample, colsample_bytree)
✅ Scale pos weight para balance de clases
✅ Penalización por overfitting >5%

Objetivo: F1 > 0.55 Y overfitting < 5%
Trials: 200
--------------------------------------------------------------------------------


  0%|          | 0/200 [00:00<?, ?it/s]

/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[I 2025-12-04 08:26:11,972] Trial 0 finished with value: -10.0 and parameters: {'num_leaves': 25, 'max_depth': 10, 'learning_rate': 0.22227824312530747, 'n_estimators': 200, 'min_child_samples': 9, 'subsample': 0.662397808134481, 'colsample_bytree': 0.6232334448672797, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 6.011150117432088}. Best is trial 0 with value: -10.0.


...

[I 2025-12-04 08:26:41,537] Trial 195 finished with value: -10.0 and parameters: {'num_leaves': 20, 'max_depth': 5, 'learning_rate': 0.2903923348939688, 'n_estimators': 197, 'min_child_samples': 5, 'subsample': 0.8551091085527746, 'colsample_bytree': 0.7307888541894453, 'reg_alpha': 0.8507668499590457, 'reg_lambda': 6.29843412960459}. Best is trial 0 with value: -10.0.
[I 2025-12-04 08:26:41,684] Trial 196 finished with value: -10.0 and parameters: {'num_leaves': 16, 'max_depth': 4, 'learning_rate': 0.2580622253163984, 'n_estimators': 265, 'min_child_samples': 27, 'subsample': 0.6347116694898893, 'colsample_bytree': 0.8863433127079721, 'reg_alpha': 0.0016519597987456613, 'reg_lambda': 7.0222028296555115}. Best is trial 0 with value: -10.0.


/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarni

[I 2025-12-04 08:26:41,847] Trial 197 finished with value: -10.0 and parameters: {'num_leaves': 19, 'max_depth': 8, 'learning_rate': 0.18723480320664312, 'n_estimators': 214, 'min_child_samples': 26, 'subsample': 0.8965300680907637, 'colsample_bytree': 0.7983548929803128, 'reg_alpha': 0.36962721281722033, 'reg_lambda': 3.7278605548658277}. Best is trial 0 with value: -10.0.
[I 2025-12-04 08:26:41,968] Trial 198 finished with value: -10.0 and parameters: {'num_leaves': 23, 'max_depth': 3, 'learning_rate': 0.2778843893606163, 'n_estimators': 59, 'min_child_samples': 19, 'subsample': 0.8048894622074663, 'colsample_bytree': 0.8958204790846345, 'reg_alpha': 0.3038958068403591, 'reg_lambda': 1.8730626923830909}. Best is trial 0 with value: -10.0.


/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[I 2025-12-04 08:26:42,128] Trial 199 finished with value: -10.0 and parameters: {'num_leaves': 18, 'max_depth': 5, 'learning_rate': 0.03558537632447262, 'n_estimators': 203, 'min_child_samples': 21, 'subsample': 0.964490537803425, 'colsample_bytree': 0.7781873392656515, 'reg_alpha': 0.542459668792896, 'reg_lambda': 0.8657652446942907}. Best is trial 0 with value: -10.0.

✅ Optimización completada


## 7. Evaluación del Mejor Modelo


In [7]:
# Entrenar mejor modelo
best_params = study.best_params
scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])

# Construir parámetros completos
final_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': best_params['num_leaves'],
    'max_depth': best_params['max_depth'],
    'learning_rate': best_params['learning_rate'],
    'n_estimators': best_params['n_estimators'],
    'min_child_samples': best_params['min_child_samples'],
    'subsample': best_params['subsample'],
    'colsample_bytree': best_params['colsample_bytree'],
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
    'scale_pos_weight': scale_pos_weight,
    'random_state': 42,
    'verbose': -1,
    'n_jobs': -1
}

best_model = lgb.LGBMClassifier(**final_params)
best_model.fit(X_train_tfidf_dense, y_train)
results = evaluate_model(best_model, X_train_tfidf_dense, X_test_tfidf_dense, y_train, y_test)

print("="*80)
print("RESULTADOS FINALES - LIGHTGBM")
print("="*80)
print(f"F1-score (test): {results['test_f1']:.4f}")
print(f"Accuracy (test): {results['test_accuracy']:.4f}")
print(f"Precision (test): {results['test_precision']:.4f}")
print(f"Recall (test): {results['test_recall']:.4f}")
print(f"Diferencia F1: {results['diff_f1']:.2f}%")
print(f"\nMatriz de confusión:")
print(results['confusion_matrix'])

if results['diff_f1'] < 5.0 and results['test_f1'] > 0.55:
    print("\n✅✅✅ OBJETIVO CUMPLIDO: Overfitting < 5% Y F1 > 0.55")
elif results['diff_f1'] < 6.0:
    print("\n🎯 MUY CERCA: Overfitting < 6%")
else:
    print("\n⚠️  Overfitting aún alto")

print("="*80)


RESULTADOS FINALES - LIGHTGBM
F1-score (test): 0.4868
Accuracy (test): 0.5150
Precision (test): 0.4742
Recall (test): 0.5000
Diferencia F1: 35.41%

Matriz de confusión:
[[57 51]
 [46 46]]

⚠️  Overfitting aún alto


/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


## 8. Validación Cruzada


In [8]:
X_all = np.vstack([X_train_tfidf_dense, X_test_tfidf_dense])
y_all = np.concatenate([y_train, y_test])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(best_model, X_all, y_all, cv=cv, scoring='f1', n_jobs=-1)

print(f"F1-score (CV): {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"Scores: {cv_scores}")


/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarni

F1-score (CV): 0.4675 (+/- 0.1141)
Scores: [0.50717703 0.55026455 0.45454545 0.44186047 0.38372093]


/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/barbara/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarni

## 9. Guardar Modelo (si cumple objetivos)


In [9]:
if results['diff_f1'] < 6.0 and results['test_f1'] > 0.55:
    with open('../models/lightgbm_model.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    with open('../models/lightgbm_tfidf.pkl', 'wb') as f:
        pickle.dump(tfidf, f)
    
    model_info = {
        'model_type': 'LightGBM',
        'hyperparameters': final_params,
        'test_f1': results['test_f1'],
        'diff_f1': results['diff_f1'],
        'cv_f1_mean': cv_scores.mean(),
        'data_augmentation': False
    }
    
    with open('../models/lightgbm_info.pkl', 'wb') as f:
        pickle.dump(model_info, f)
    
    print("✅ Modelo LightGBM guardado")
else:
    print("⚠️  Modelo no guardado (no cumple objetivos)")


⚠️  Modelo no guardado (no cumple objetivos)


## 10. Análisis de Feature Importance


In [10]:
# Feature importance (top 20)
feature_names = tfidf.get_feature_names_out()
importances = best_model.feature_importances_
indices = np.argsort(importances)[::-1][:20]

print("Top 20 features más importantes:")
print("-"*50)
for i in range(20):
    print(f"{i+1:2d}. {feature_names[indices[i]]:30s} {importances[indices[i]]:.4f}")


Top 20 features más importantes:
--------------------------------------------------
 1. people                         186.0000
 2. police                         108.0000
 3. shoot                          69.0000
 4. cop                            68.0000
 5. black                          59.0000
 6. white                          56.0000
 7. need                           55.0000
 8. like                           52.0000
 9. brown                          51.0000
10. say                            51.0000
11. fuck                           48.0000
12. man                            39.0000
13. guy                            38.0000
14. know                           37.0000
15. shit                           37.0000
16. officer                        36.0000
17. let                            35.0000
18. come                           34.0000
19. time                           33.0000
20. law                            32.0000
